<header>
    <p style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color:#00233c; padding-left:20pt; padding-top:20pt; padding-bottom:10pt; padding-right:20pt;'>
        Teradata MCP – Conversational Interaction<br>
        <span style="font-size:24px;">02 – Chat with MCP and Teradata Vantage</span><br>
        <img src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width:125px; height:auto; margin-top:20pt;">
    </p>
</header>

<p style="font-size:20px;font-family:Arial;"><b>Introduction</b></p>

<p style="font-size:16px;font-family:Arial;">
The <b>Teradata Model Context Protocol (MCP)</b> allows AI agents to interact with data systems through a secure, contextual interface.  
In this notebook, we demonstrate how MCP integrates with <b>ChatMCPController</b> to enable a conversational AI that can access Teradata Vantage in real time.
</p>

<p style="font-size:16px;font-family:Arial;">
This demonstration simulates a scenario where an enterprise data steward or business analyst interacts with an AI assistant to explore databases, summarize information, and generate executive insights — all without writing SQL.
</p>

<p style="font-size:18px;font-family:Arial;"><b>Key Highlights</b></p>
<ul style="font-size:16px;font-family:Arial;">
<li>Spin up the <b>Teradata MCP Community Server</b> connected to Vantage.</li>
<li>Securely pass database credentials and configure the runtime environment.</li>
<li>Launch an interactive chat interface using <b>ChatMCPController</b>.</li>
<li>Enable natural-language interaction with enterprise data and tools.</li>
<li>Stop and clean up MCP servers at the end of the session.</li>
</ul>

<hr style="height:2px;border:none;">

<p style="font-size:16px;font-family:Arial;">
This notebook builds on the foundation set in <b>01-Explore-MCP-Servers.ipynb</b> and takes the next step in the Teradata MCP journey — enabling conversational interaction between AI agents and Teradata Vantage.  
It demonstrates how to spin up the MCP Community Server, connect it to Vantage, and then use <code>ChatMCPController</code> to simulate agentic AI conversations that access and reason over enterprise data.
</p>

<p style="font-size:16px;font-family:Arial;">
With this, you’ll witness how AI-driven analytics can query, interpret, and summarize Teradata data through natural language — a key capability in <b>Agentic AI applications</b> powered by the <b>Model Context Protocol (MCP)</b>.
</p>

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>if you had opened the notebook before doing the pip installs in the setup notebook, now it's a good moment to restart the notebook's kernel</i></p>
</div>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>1. Connect to Vantage and Import packages</b></p>

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

from teradataml import *
import os
import getpass
import chatbot_widget
from chatbot_widget.mcp.server_manager import MCPServerManager

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'> 1.1 Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
#Connection
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=02-chat_with_MCP.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;">
<p style="font-size:20px;font-family:Arial;"><b>2. Initialize and Start the MCP Server</b></p>
<p style="font-size:16px;font-family:Arial;">
Now that the environment is connected, we instantiate the <b>MCPServerManager</b> from the <code>chatbot_widget.mcp</code> package.  
This utility will start the <b>Teradata MCP Community Server</b>, which acts as a gateway between the conversational AI and Vantage.
</p>

<p style="font-size:18px;font-family:Arial;"><b>2.1 Initialize MCP Server Manager</b></p>
<p style="font-size:16px;font-family:Arial;">
We create an <code>MCPServerManager</code> instance to manage the lifecycle of our servers — start, check health, stop, or restart.
</p>

In [ ]:
mcp = MCPServerManager()

<p style="font-size:18px;font-family:Arial;"><b>2.2 Start Teradata MCP Community Server</b></p>
<p style="font-size:16px;font-family:Arial;">
We start the official <b>Teradata MCP Community Server</b> (<code>tdmcpcommunity.py</code>) to bridge our AI chat controller with Teradata Vantage.</p>

In [ ]:
mcp.start(
    "tdmcpcommunity",
    os.path.abspath("tdmcpcommunity.py"),
    port=None,  # or fixed port like 8765
    host="host.docker.internal",
    user=username,
    password=password
)

<p style="font-size:18px;font-family:Arial;"><b>2.3 Verify MCP Server Health</b></p>
<p style="font-size:16px;font-family:Arial;">
After waiting for a few seconds, we check the server’s health to confirm that it’s active and ready to handle requests.
</p>

In [ ]:
#check after 15 seconds, DB connection needs to start
mcp.check_health("tdmcpcommunity")

<hr style="height:2px;border:none;">
<p style="font-size:20px;font-family:Arial;"><b>3. Launch the Chat Interface</b></p>
<p style="font-size:16px;font-family:Arial;">
Once the MCP Server is running, we connect it to an AI language model using the <b>ChatMCPController</b>.  
This controller orchestrates a conversational interface that allows users to query the Teradata environment using natural language.
</p>

<p style="font-size:18px;font-family:Arial;"><b>3.1 Configure OpenAI API Key</b></p>
<p style="font-size:16px;font-family:Arial;">
We use <code>getpass()</code> to securely set the <code>OPENAI_API_KEY</code> environment variable, which is required to access the underlying LLM (e.g., GPT-5).
</p>

In [ ]:
# Enter the Open AI Key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

<p style="font-size:18px;font-family:Arial;"><b>3.2 Initialize Chat Controller</b></p>
<p style="font-size:16px;font-family:Arial;">
The <code>ChatMCPController</code> connects the LLM to the MCP Server, enabling it to use available tools and context when responding to queries.
</p>

In [ ]:
from chatbot_widget.controller.chat_mcp_controller import ChatMCPController
# you can ignore the warnings
mychat = ChatMCPController(mcp)

<p style="font-size:18px;font-family:Arial;"><b>3.3 Launch Interactive Chat</b></p>
<p style="font-size:16px;font-family:Arial;">
We launch the chat interface directly inside the notebook.  
Users can begin conversations using simple commands or contextual questions such as:
</p>

<ul style="font-size:16px;font-family:Arial;">
<li><code>/tools</code> – List all MCP tools the AI agent can access.</li>
<li><i>"What tables are in my demo_user database?"</i></li>
<li><i>"Are there any data quality issues with payments?"</i></li>
<li><i>"Give me a 300-word executive summary of what analytics can be done on my payments table."</i></li>
</ul>

<div class="alert alert-block alert-info">
<p style="font-size:16px;font-family:Arial;">
<b>Note:</b><i> This interface is a prototype intended for exploration.  
Response generation may take time, and intermediate progress indicators are currently not displayed.
</i></p>
</div>

In [ ]:
mychat.display()

<hr style="height:2px;border:none;">
<p style="font-size:20px;font-family:Arial;"><b>4. Manage and Stop MCP Servers</b></p>
<p style="font-size:16px;font-family:Arial;">
Once all operations are complete, stop all running MCP servers to free system resources.
</p>

<p style="font-size:18px;font-family:Arial;"><b>4.1 Stop All Servers</b></p>
<p style="font-size:16px;font-family:Arial;">
Call <code>mcp.stop_all()</code> to close all active sessions and confirm proper shutdown.
</p>

In [ ]:
mcp.stop_all()

<hr style="height:2px;border:none;">
<p style="font-size:20px;font-family:Arial;"><b>5. Conclusion</b></p>
<p style="font-size:16px;font-family:Arial;">
In this notebook, we successfully connected an AI chat interface to <b>Teradata Vantage</b> via the <b>Teradata MCP</b>.  
By leveraging the <b>ChatMCPController</b>, we demonstrated how AI agents can:
</p>

<ul style="font-size:16px;font-family:Arial;">
<li>Access Teradata data contextually through natural language queries.</li>
<li>Utilize MCP tools for metadata discovery and data exploration.</li>
<li>Generate business-ready insights through LLM reasoning over Vantage data.</li>
</ul>

<p style="font-size:16px;font-family:Arial;">
This concludes the final part of the <b>Teradata MCP Demonstration Series</b>, highlighting the power of conversational AI built on top of enterprise-grade analytics infrastructure.
</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>6. Cleanup</b></p>
<p style="font-size:16px;font-family:Arial;">
Finally, we remove the active database context using <code>remove_context()</code>.  
This ensures that all database connections are safely closed and resources are released.
</p>

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>